## **db**

- https://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.html#executing-with-an-orm-db
- https://www.sqlalchemy.org/

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv, find_dotenv
import os
import sqlalchemy
import os
from sqlalchemy.engine import Engine
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine, text, Column, Integer, String, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.exc import OperationalError, DisconnectionError
import time


In [3]:
_ = load_dotenv(find_dotenv())

In [4]:
_

True

In [5]:
# Get connection string from environment variable
conn_str = os.environ.get('CONNECTION_STRING')

# Optional: Handle missing connection string
if not conn_str:
    raise ValueError("CONNECTION_STRING environment variable not set.")

# Create SQLAlchemy engine
engine: Engine = create_engine(conn_str,
    echo=False,
    pool_size=10,
    max_overflow=20,
    pool_pre_ping=True,  # Verify connection is alive before use
    pool_recycle=3600,   # Reconnect after 1 hour (helps with Neon timeouts)
)

In [6]:
# --- Step 2: Define Base & Models (as provided previously) ---
Base = declarative_base()

class Department(Base):
    __tablename__ = 'departments'
    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True, nullable=False)
    employees = relationship('Employee', back_populates='department', cascade="all, delete-orphan") # Added cascade for robust deletion

    def __repr__(self):
        return f"<Department(id={self.id}, name='{self.name}')>"

class Employee(Base):
    __tablename__ = 'employees'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    department_id = Column(Integer, ForeignKey('departments.id'))
    department = relationship('Department', back_populates='employees')

    def __repr__(self):
        return f"<Employee(id={self.id}, name='{self.name}', department_id={self.department_id})>"


A one-to-many relationship in an ORM (Object-Relational Mapper) like SQLAlchemy describes a common database pattern where a single record in one table can be associated with multiple records in another table. For example, one author can write many books, or one department can have many employees.

-----

##### Understanding One-to-Many

In a one-to-many relationship:

  * **"One" side (Parent):** This entity holds the primary key.
  * **"Many" side (Child):** This entity holds a **foreign key** that references the primary key of the "one" side.

Consider the example of `Department` and `Employee`:

  * A `Department` can have many `Employee`s.
  * Each `Employee` belongs to exactly one `Department`.


##### Explanation of Key Components:

  * **`ForeignKey('departments.id')`**: This is defined on the `Employee` model (`department_id` column). It tells SQLAlchemy that this column refers to the `id` column of the `departments` table. This is the **crucial part for the database relationship**.
  * **`relationship('Employee', back_populates='department')`** (on `Department`):
      * `'Employee'` refers to the Python class name of the related model.
      * `back_populates='department'` tells SQLAlchemy that there's a corresponding `relationship()` on the `Employee` model named `department` that points back to `Department`. This creates a **bidirectional relationship**, allowing you to navigate from `Department` to `Employee`s (`department.employees`) and from an `Employee` to their `Department` (`employee.department`).
  * **`relationship('Department', back_populates='employees')`** (on `Employee`):
      * `'Department'` refers to the Python class name of the related model.
      * `back_populates='employees'` tells SQLAlchemy that there's a corresponding `relationship()` on the `Department` model named `employees`.

In [7]:
# --- Step 3: Create tables (Run this once, e.g., on application start or migration) ---
Base.metadata.create_all(bind=engine)
print("Database tables created/ensured.")

Database tables created/ensured.


In [8]:
# --- Step 4: Set up db (Session local factory and instance) ---
dbmaker = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Function to safely get a session, with retry logic for connection errors
def get_db():
    db = dbmaker()
    try:
        yield db # This makes it a context manager / generator
    finally:
        db.close()

In [9]:
# --- Populate Initial Data (run this only once, or handle existing data) ---
# It's good practice to ensure data population is idempotent
# For a real app, you might check if data exists before adding
with next(get_db()) as db: # Use get_db() to create and manage session
    if not db.query(Department).first(): # Only add if no departments exist
        print("\nPopulating initial data...")
        hr_dept = Department(name='Human Resources')
        it_dept = Department(name='Information Technology')
        db.add_all([hr_dept, it_dept])
        db.commit()

        # Re-fetch departments to ensure they have IDs if db.add_all didn't refresh them
        hr_dept = db.query(Department).filter_by(name='Human Resources').first()
        it_dept = db.query(Department).filter_by(name='Information Technology').first()

        db.add_all([
            Employee(name='Alice', department=hr_dept),
            Employee(name='Bob', department=hr_dept),
            Employee(name='Charlie', department=it_dept)
        ])
        db.commit()
        print("Initial data populated.")
    else:
        print("\nInitial data already exists. Skipping population.")


Populating initial data...
Initial data populated.


In [10]:
# --- Database Operations with Error Handling ---

# Helper function to retry operations on connection errors
def retry_db_operation(operation, max_retries=3, delay_seconds=1):
    for i in range(max_retries):
        try:
            with next(get_db()) as db: # Get a fresh session for each attempt
                db_session = operation(db)
                return db_session
        except (OperationalError, DisconnectionError) as e:
            print(f"Database connection error (Attempt {i+1}/{max_retries}): {e}")
            if i < max_retries - 1:
                time.sleep(delay_seconds * (2**i)) # Exponential backoff
            else:
                raise # Re-raise if all retries fail

In [11]:
# 3. Querying from the "One" side (Department -> Employees)
print("\n--- Querying Department and its Employees ---")
def query_hr_employees(db_session):
    retrieved_hr = db_session.query(Department).filter_by(name='Human Resources').first()
    if retrieved_hr:
        print(f"\nDepartment: {retrieved_hr.name}")
        for emp in retrieved_hr.employees:
            print(f"  - Employee: {emp.name}")
    return retrieved_hr # Return for potential further use

try:
    retry_db_operation(query_hr_employees)
except Exception as e:
    print(f"Failed to query HR department after multiple retries: {e}")


--- Querying Department and its Employees ---

Department: Human Resources
  - Employee: Alice
  - Employee: Bob


In [12]:
# 4. Querying from the "Many" side (Employee -> Department)
print("\n--- Querying Employee and their Department ---")
def query_alice_department(db_session):
    retrieved_emp = db_session.query(Employee).filter_by(name='Alice').first()
    if retrieved_emp:
        print(f"\nEmployee: {retrieved_emp.name}")
        print(f"  - Belongs to Department: {retrieved_emp.department.name}")
    return retrieved_emp

try:
    retry_db_operation(query_alice_department)
except Exception as e:
    print(f"Failed to query Alice's department after multiple retries: {e}")


--- Querying Employee and their Department ---

Employee: Alice
  - Belongs to Department: Human Resources


In [13]:
# 5. Adding an Employee to an existing Department (after initial creation)
print("\n--- Adding an Employee to an Existing Department ---")
def add_david_to_it(db_session):
    new_employee = Employee(name='David')
    retrieved_it = db_session.query(Department).filter_by(name='Information Technology').first()
    if retrieved_it:
        # Check if David already exists in this department to prevent duplicates in example
        if not any(e.name == 'David' for e in retrieved_it.employees):
            retrieved_it.employees.append(new_employee)
            db_session.add(new_employee)
            db_session.commit()
            print(f"David added to {retrieved_it.name}. Employees in IT now:")
            # Re-fetch or refresh to see the change in the relationship collection
            db_session.refresh(retrieved_it)
            for emp in retrieved_it.employees:
                print(f"  - {emp.name}")
        else:
            print("David already exists in IT department.")
    else:
        print("Information Technology department not found.")
    return retrieved_it

try:
    retry_db_operation(add_david_to_it)
except Exception as e:
    print(f"Failed to add David after multiple retries: {e}")


--- Adding an Employee to an Existing Department ---
David added to Information Technology. Employees in IT now:
  - Charlie
  - David


In [14]:
# 6. Deleting an Employee
print("\n--- Deleting an Employee ---")
def delete_bob(db_session):
    emp_to_delete = db_session.query(Employee).filter_by(name='Bob').first()
    if emp_to_delete:
        db_session.delete(emp_to_delete)
        db_session.commit()
        print(f"Bob deleted.")
        retrieved_hr = db_session.query(Department).filter_by(name='Human Resources').first()
        if retrieved_hr:
            db_session.refresh(retrieved_hr) # Refresh to see updated employee list
            print(f"Employees in HR now:")
            if retrieved_hr.employees:
                for emp in retrieved_hr.employees:
                    print(f"  - {emp.name}")
            else:
                print("  (No employees in HR)")
    else:
        print("Employee 'Bob' not found.")
    return emp_to_delete

try:
    retry_db_operation(delete_bob)
except Exception as e:
    print(f"Failed to delete Bob after multiple retries: {e}")


--- Deleting an Employee ---
Bob deleted.
Employees in HR now:
  - Alice


In [15]:
# Query for an employee named "Alice" (re-run as a separate operation)
print("\n--- Final Check for Alice ---")
def final_check_alice(db_session):
    alice = db_session.query(Employee).filter_by(name='Alice').first()
    if alice:
        print(f"Found Employee: {alice}")
    else:
        print("Employee 'Alice' not found.")
    non_existent_employee = db_session.query(Employee).filter_by(name='Zoe').first()
    if non_existent_employee:
        print(f"Found Employee: {non_existent_employee}")
    else:
        print("\nEmployee 'Zoe' not found (as expected).")

try:
    retry_db_operation(final_check_alice)
except Exception as e:
    print(f"Failed final check for Alice after multiple retries: {e}")


--- Final Check for Alice ---
Found Employee: <Employee(id=1, name='Alice', department_id=1)>

Employee 'Zoe' not found (as expected).
